In [72]:
# All query results are returned as Pandas DataFrames 
import pandas as pd

# Needed for %sql Magic
%load_ext sql

import sqlite3

conn = sqlite3.connect("./CourseDataWarehouseFinal.db")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [73]:
%sql sqlite:///CourseDataWarehouseFinal.db

'Connected: @CourseDataWarehouseFinal.db'

In [74]:
%%sql
DROP TABLE IF EXISTS dimCourses;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

In [75]:
%%sql
DROP TABLE IF EXISTS dimCap;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

In [76]:
%%sql
DROP TABLE IF EXISTS dimTerm;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

In [77]:
%%sql
DROP TABLE IF EXISTS dimCRN;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

In [78]:
%%sql

DROP TABLE IF EXISTS dimCourses;

CREATE TABLE dimCourses(
    program_code VarChar NOT NULL,
    catalog_id VarChar NOT NULL,
    course_title TEXT NOT NULL,
    credits INT NOT NULL,
    prereqs VarChar,
    coreqs VarChar,
    fees VarChar,
    attributes TEXT,
    description TEXT,
    PRIMARY KEY (program_code, catalog_id)
);
    
INSERT INTO dimCourses (program_code, catalog_id, course_title, credits, prereqs, coreqs, fees, attributes, description)
SELECT DISTINCT program_code, catalog_id, course_title, credits, prereqs, coreqs, fees, attributes, description
FROM CourseCatalog;


 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
2220 rows affected.


[]

In [79]:
%%sql

DROP TABLE IF EXISTS dimTerm;

CREATE TABLE dimTerm(
    term VarChar NOT NULL,
    title TEXT NOT NULL,
    crn INT NOT NULL,
    credits INT NOT NULL,
    Location VarChar,
    Primary_Instructor TEXT NOT NULL,
    PRIMARY KEY (crn, term)
);

INSERT INTO dimTerm(term, title, crn, credits, Location, primary_instructor)
SELECT DISTINCT term, title, crn, credits, Location, primary_instructor
FROM courses_final;

 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
13860 rows affected.


[]

In [80]:
%%sql

SELECT count(DISTINCT Primary_Instructor)
FROM dimTerm;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


count(DISTINCT Primary_Instructor)
1085


In [81]:
%%sql
DROP TABLE IF EXISTS dimLocation;

CREATE TABLE dimLocation(
    Location VarChar,
    Primary_Instructor TEXT NOT NULL,
    title TEXT NOT NULL,
    term VarChar NOT NULL,
    act INT NOT NULL,
    cap INT NOT NULL,
    rem INT NOT NULL
);

INSERT INTO dimLocation(Location, Primary_Instructor, title, term, act, cap, rem)
SELECT DISTINCT Location, Primary_Instructor, title, term, act, cap, rem
FROM courses_final;

 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
13251 rows affected.


[]

In [82]:
%%sql
select *
FROM dimLocation
LIMIT 15;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


Location,Primary_Instructor,title,term,act,cap,rem
None,Lauren Elise Cesiro,History of Photography,Winter2017,18,20,2
None,Thomas B. Cunningham,Ecology and Society,Winter2017,21,20,-1
None,Thomas B. Cunningham,Ecology and Society,Winter2017,19,20,1
DSB109,Sharlene A. McEvoy,Legal Environment of Business,Winter2017,20,25,5
None,David Gudelunas,Human Communication Theories,Winter2017,19,20,1
None,Qin Zhang,Intercultural Communication,Winter2017,16,20,4
DMH350,Margaret A. Wills,Family Communication,Winter2017,15,15,0
None,William F. Vasquez Mazariegos,Introduction to Macroeconomics,Winter2017,12,20,8
None,Gita Rajan,Introduction to Literary and Cultural Studies,Winter2017,8,20,12
None,Jacqueline Rinaldi,The Literature of Illness and Healing: Wounded Storytellers and Dedicated Healers,Winter2017,17,20,3


In [83]:
%%sql

DROP TABLE IF EXISTS dimDate;

CREATE TABLE dimDate(
    term VarChar NOT NULL,
    StartDate DATE NOT NULL,
    EndDate DATE NOT NULL    
);

INSERT INTO dimDate(term, StartDate, EndDate)
select term, StartDate, EndDate
FROM terms;

 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
16 rows affected.


[]

In [84]:
%%sql
select *
from dimDate;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


term,StartDate,EndDate
Fall2014,2014-08-16,2014-12-29
Fall2015,2015-08-15,2015-12-19
Fall2016,2016-08-13,2016-12-19
Fall2018,2018-08-11,2018-12-17
Spring2015,2015-01-10,2015-05-16
Spring2016,2016-01-13,2016-05-16
Spring2017,2017-01-11,2017-05-27
Spring2018,2018-01-10,2018-05-15
Spring2019,2019-01-22,2019-05-15
SpringBreak2017,2017-03-13,2017-03-17


In [85]:
%%sql

DROP TABLE IF EXISTS factClass;

CREATE TABLE factClass ( 
    program_code VarChar,
    Term VarChar,
    Location VarChar,
    StartDate Date,
    EndDate Date,
    FOREIGN KEY (program_code) REFERENCES dimCourses(program_code),
    FOREIGN KEY (Term) REFERENCES dimTerm(Term),
    FOREIGN KEY (Location) REFERENCES dimLocation(Location),
    FOREIGN KEY (StartDate) REFERENCES dimDate(StartDate),  
    FOREIGN KEY (EndDate) REFERENCES dimDate(EndDate)
);

INSERT INTO factClass (program_code, Term, Location, StartDate, EndDate)
SELECT DISTINCT dimCourses.program_code, dimTerm.Term, dimLocation.Location, dimDate.StartDate, dimDate.EndDate
FROM dimCourses
LEFT JOIN dimTerm ON dimCourses.course_title = dimTerm.title
LEFT JOIN dimLocation ON dimTerm.Location = dimLocation.Location
LEFT JOIN dimDate ON dimLocation.Term = dimDate.Term;


 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
56704 rows affected.


[]

In [86]:
%%sql

SELECT *
FROM factClass
LIMIT 15;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


program_code,Term,Location,StartDate,EndDate
AC,Fall2014,DSB105,2014-08-16,2014-12-29
AC,Fall2014,DSB105,2015-08-15,2015-12-19
AC,Fall2014,DSB105,2016-08-13,2016-12-19
AC,Fall2014,DSB105,2018-08-11,2018-12-17
AC,Fall2014,DSB105,2015-01-10,2015-05-16
AC,Fall2014,DSB105,2016-01-13,2016-05-16
AC,Fall2014,DSB105,2017-01-11,2017-05-27
AC,Fall2014,DSB105,2018-01-10,2018-05-15
AC,Fall2014,DSB105,2019-01-22,2019-05-15
AC,Fall2014,DSB105,2016-04-13,2016-08-27
